In [44]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [45]:
pip install networkx

Note: you may need to restart the kernel to use updated packages.


In [103]:
import pandas as pd
import networkx as nx
from math import radians, cos, sin, asin, sqrt
import re


In [47]:
# Chargement du fichier timetables.csv
df_timetables = pd.read_csv('timetables.csv', sep='\t', on_bad_lines='skip')

# Affichage des premières lignes pour vérifier que le fichier a bien été chargé
print(df_timetables.head())

                 trip_id                                             trajet  \
0  OCESN003100F140147152         Gare de Le Havre - Gare de Paris-St-Lazare   
1  OCESN003190F040047309           Gare de Dieppe - Gare de Paris-St-Lazare   
2  OCESN003198F030037315  Gare de Paris-St-Lazare - Gare de Rouen-Rive-D...   
3  OCESN003300F030037323        Gare de Cherbourg - Gare de Paris-St-Lazare   
4  OCESN003313F380387526             Gare de Caen - Gare de Paris-St-Lazare   

   duree  
0    138  
1    145  
2     97  
3    194  
4    149  


In [93]:
# Fonction pour nettoyer les noms 
def nettoyer_nom_gare(nom):
    # Remplacer les tirets multiples par un seul
    nom = re.sub(r'-+', '-', nom)

    # Supprimer les parenthèses et leur contenu
    nom = re.sub(r'\(.*?\)', '', nom).strip()

    # Supprimer les espaces multiples
    nom = re.sub(r'\s+', ' ', nom)

    # Corriger les cas où il y a un tiret à la fin du nom
    if nom.endswith('-'):
        nom = nom.rstrip('-').strip()

    return nom

In [94]:
# Fonction pour découper les trajets avec un traitement spécial pour les noms composés
def decouper_trajet(trajet):
    # Découper en deux parties entre les gares uniquement, en séparant par " - "
    villes = trajet.rsplit(' - ', 1) 
    
    if len(villes) == 2:
        depart = nettoyer_nom_gare(villes[0])
        arrivee = nettoyer_nom_gare(villes[1])
        return depart, arrivee
    else:
        print(f"Erreur : Impossible de découper correctement le trajet {trajet}")
        return None, None

In [97]:
# Fonction pour construire le graphe à partir du CSV
def construire_graphe_avec_csv(df_timetables):
    G = nx.Graph()

    # Boucle sur toutes les lignes du fichier timetables.csv
    for index, row in df_timetables.iterrows():
        trajet = row['trajet']
        duree = row['duree']  # Récupérer la durée du trajet

        # Découper le trajet en gare de départ et gare d'arrivée
        depart, arrivee = decouper_trajet(trajet)

        if depart and arrivee:
            # Ajouter les gares comme nœuds et les trajets comme arêtes
            G.add_node(depart)  # Ajouter la gare de départ comme nœud
            G.add_node(arrivee)  # Ajouter la gare d'arrivée comme nœud
            G.add_edge(depart, arrivee, weight=duree)  # Ajouter l'arête avec la durée comme poids
        else:
            print(f"Impossible de traiter le trajet {trajet}")

    return G

# Création du graphe à partir du CSV
graphe = construire_graphe_avec_csv(df_timetables)

# Vérifier que le graphe a bien été construit
print("Gares dans le graphe :", list(graphe.nodes()))

Gares dans le graphe : ['Gare de Le Havre', 'Gare de Paris-St-Lazare', 'Gare de Dieppe', 'Gare de Rouen-Rive-Droite', 'Gare de Cherbourg', 'Gare de Caen', 'Gare de Trouville-Deauville', 'Gare de Granville', 'Gare de Paris-Montp.3-Vaug.', 'Gare de Paris-Montparnasse 1-2', 'Gare de Dreux', 'Gare de Bourges', 'Gare de Paris-Austerlitz', 'Gare de Vierzon', 'Gare de Paris-Bercy', 'Gare de Nevers', 'Gare de Paris-Gare-de-Lyon', 'Gare de Argentan', 'Gare de Orléans', 'Gare de Tours', 'Gare de Aubrais', 'Gare de St-Pierre-des-Corps', 'Gare de Calais Ville', 'Gare de Paris Gare du Nord', 'Gare de Boulogne Ville', 'Gare de Amiens', 'Gare de Cambrai Ville', 'Gare de Maubeuge', 'Gare de St Quentin', 'Gare de Compiègne', 'Gare de Le Mans', 'Gare de Nogent-le-Rotrou', 'Gare de Chartres', 'Gare de Blois-Chambord', 'Gare de Lyon-Perrache', 'Gare de Latour-de-Carol-Enveitg', 'Gare de Villefranche-Vernet-l.B', 'Gare de Font-Romeu-Odeillo-Via', 'Gare de Briançon', 'Gare de Romans-Bourg-de-Péage', 'Gare d

In [98]:
# Fonction pour calculer la durée totale d'un chemin
def calculer_duree_totale(G, chemin):
    duree_totale = 0
    for i in range(len(chemin) - 1):
        # Récupérer la durée (poids) de l'arête entre deux gares consécutives
        duree = G[chemin[i]][chemin[i + 1]]['weight']
        duree_totale += duree
    return duree_totale

In [99]:
# Fonction pour calculer le chemin le plus court entre deux gares avec A* et la durée totale
def trouver_chemin_le_plus_court(G, ville_depart, ville_arrivee): 
    if ville_depart in G.nodes() and ville_arrivee in G.nodes():
        # Trouver le chemin avec A*
        chemin = nx.astar_path(G, ville_depart, ville_arrivee, weight='weight')
        # Calculer la durée totale du chemin
        duree_totale = calculer_duree_totale(G, chemin)
        return chemin, duree_totale
    else:
        print(f"L'une des villes {ville_depart} ou {ville_arrivee} n'est pas présente dans le graphe.")
        return None, None

In [104]:
# Exemple : Trouver le chemin entre Lyon et Paris avec la durée totale
chemin_optimal, duree_totale = trouver_chemin_le_plus_court(graphe, 'Gare de Rennes', 'Gare de Marseille-St-Charles')

print("Chemin optimal :", chemin_optimal)
print("Durée totale du chemin :", duree_totale, "minutes")

Chemin optimal : ['Gare de Rennes', 'Gare de Nantes', 'Gare de Tours', 'Gare de Nevers', 'Gare de Lyon-Perrache', 'Gare de Valence-Ville', 'Gare de Marseille-St-Charles']
Durée totale du chemin : 765 minutes
